<a href="https://colab.research.google.com/github/Muhammadtriiki/homework-repo/blob/main/projects/Build_a_Python_Website_in_15_Minutes_With_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a Python Website in 15 Minutes With Streamlit

To begin, you'll need VS Code or any local code editor to work with the following code. The implementation has been carefully reviewed and enhanced with detailed comments for better understanding.

This code creates an interactive website using Python and Streamlit, featuring a data dashboard with various analysis and visualization capabilities. I've enhanced the basic functionality to make it more user-friendly and effective while maintaining the core requirements. The code is organized into modular functions for better readability and includes basic CSS styling for visual appeal.

## Required Libraries
Before running the code, install these dependencies:
```bash
pip install streamlit
pip install pandas
pip install plotly
pip install seaborn
pip install matplotlib
```

## Key Features
1. Interactive data upload interface
2. Basic data overview and statistics
3. Column-wise filtering and analysis
4. Multiple visualization options:
   - Correlation heatmaps
   - Scatter plots
   - Histograms
   - Box plots
5. Missing value analysis
6. Data download functionality

## Testing the Application
To test this dashboard, you'll need a CSV file. You can download sample datasets from this GitHub repository:
[Sample Datasets Repository](https://github.com/techwithtim/Streamlit-Intro-App/blob/main/Sample_Data_for_Plotting_and_Filtering.csv)

## Running the Application
After saving the code, run it using:
```bash
streamlit run file.py
or
python -m streamlit run file.py

```

The code is thoroughly commented for easy understanding, with each function having a specific purpose in the data analysis pipeline. Whether you're a beginner or experienced developer, you should be able to follow the implementation and customize it for your needs.

Would you like me to explain any specific part of the code in more detail?

In [ ]:
# Import required libraries
import pandas as pd              # For data manipulation and analysis
import streamlit as st          # For creating the web application
import plotly.express as px     # For interactive plotting
import seaborn as sns          # For statistical data visualization
import matplotlib.pyplot as plt # For creating static plots

# Configure the Streamlit page settings
# - Sets page title in browser tab
# - Uses wide layout for better visualization
# - Keeps sidebar expanded by default
st.set_page_config(page_title="Data Dashboard", layout="wide", initial_sidebar_state="expanded")

# Add custom CSS styling to the application
# - Sets background gradient
# - Defines custom font size for headers
st.markdown("""
    <style>
    .stApp {
        background: linear-gradient(135deg, #f5f7fa 0%, #e8ecf2 100%);
    }
    .big-font {
        font-size: 24px !important;
        font-weight: bold;
    }
    </style>
""", unsafe_allow_html=True)

def load_data():
    """
    Handles the file upload functionality and data loading
    Returns:
        pandas.DataFrame or None: Returns DataFrame if file is successfully loaded, None otherwise
    """
    # Display dashboard title with custom styling
    st.markdown('<p class="big-font">📊 Interactive Data Dashboard</p>', unsafe_allow_html=True)

    # Create file uploader widget that accepts only CSV files
    file_upload = st.file_uploader("Upload your CSV file", type="csv")

    # Process the uploaded file if one is provided
    if file_upload is not None:
        try:
            # Attempt to read the CSV file into a pandas DataFrame
            df = pd.read_csv(file_upload)
            st.success("File successfully uploaded! 🎉")
            return df
        except Exception as e:
            # Display error message if file reading fails
            st.error(f"Error: {str(e)}")
            return None
    return None

def data_overview(df):
    """
    Provides basic information about the dataset
    Args:
        df (pandas.DataFrame): The input DataFrame to analyze
    """
    st.subheader("📝 Data Overview")

    # Create three columns for displaying basic statistics
    col1, col2, col3 = st.columns(3)

    # Display total number of rows
    with col1:
        st.info(f"Total Rows: {df.shape[0]}")
    # Display total number of columns
    with col2:
        st.info(f"Total Columns: {df.shape[1]}")
    # Display total number of missing values
    with col3:
        st.info(f"Missing Values: {df.isna().sum().sum()}")

    # Option to show data types of all columns
    if st.checkbox("Show Data Types"):
        st.write(df.dtypes)

def column_filter(df):
    """
    Allows users to select and analyze specific columns
    Args:
        df (pandas.DataFrame): The input DataFrame to filter
    """
    st.subheader("🔍 Column Analysis")

    # Multi-select widget for column selection
    selected_columns = st.multiselect("Select columns to view", df.columns)
    if selected_columns:
        # Display selected columns from the DataFrame
        st.write(df[selected_columns])

        # Option to display unique values in selected columns
        if st.checkbox("Show unique values"):
            col = st.selectbox("Select column", selected_columns)
            st.write(f"Unique values in {col}:")
            st.write(df[col].unique())

def data_visualization(df):
    """
    Creates various visualizations of the data
    Args:
        df (pandas.DataFrame): The input DataFrame to visualize
    """
    st.subheader("📈 Data Visualization")

    # Filter for numeric columns only as they're required for these visualizations
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

    if len(numeric_cols) > 0:
        # Create correlation heatmap
        if st.checkbox("Show Correlation Heatmap"):
            fig, ax = plt.subplots(figsize=(10, 6))
            sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm', ax=ax)
            st.pyplot(fig)

        # Create interactive scatter plot
        if st.checkbox("Create Scatter Plot"):
            col1, col2 = st.columns(2)
            with col1:
                x_axis = st.selectbox("Select X-axis", numeric_cols)
            with col2:
                y_axis = st.selectbox("Select Y-axis", numeric_cols)

            fig = px.scatter(df, x=x_axis, y=y_axis, title=f"{x_axis} vs {y_axis}")
            st.plotly_chart(fig)

        # Create histogram for distribution analysis
        if st.checkbox("Create Histogram"):
            col = st.selectbox("Select column for histogram", numeric_cols)
            fig = px.histogram(df, x=col, title=f"Histogram of {col}")
            st.plotly_chart(fig)

        # Create box plot for distribution and outlier analysis
        if st.checkbox("Create Box Plot"):
            col = st.selectbox("Select column for box plot", numeric_cols)
            fig = px.box(df, y=col, title=f"Box Plot of {col}")
            st.plotly_chart(fig)

def data_summary(df):
    """
    Provides statistical summary and missing value analysis
    Args:
        df (pandas.DataFrame): The input DataFrame to summarize
    """
    st.subheader("📊 Data Summary")

    # Display statistical summary of numeric columns
    if st.checkbox("Show Statistical Summary"):
        st.write(df.describe())

    # Analyze and display missing values information
    if st.checkbox("Show Missing Values Analysis"):
        missing_data = pd.DataFrame({
            'Column': df.columns,
            'Missing Values': df.isnull().sum().values,
            'Percentage': (df.isnull().sum().values / len(df) * 100).round(2)
        })
        st.write(missing_data)

def data_download(df):
    """
    Provides functionality to download the processed DataFrame
    Args:
        df (pandas.DataFrame): The DataFrame to be downloaded
    """
    st.subheader("💾 Download Processed Data")

    # Create download button for the DataFrame
    if st.button("Generate Download Link"):
        csv = df.to_csv(index=False)
        st.download_button(
            label="Download CSV",
            data=csv,
            file_name="processed_data.csv",
            mime="text/csv"
        )

def main():
    """
    Main function that controls the flow of the application
    """
    # Create sidebar navigation
    st.sidebar.title("Navigation")

    # Load data using the file uploader
    df = load_data()

    # If data is successfully loaded, show navigation options
    if df is not None:
        # Radio buttons for page selection
        page = st.sidebar.radio("Select a page",
            ["Data Overview", "Column Analysis", "Visualization", "Summary", "Download"])

        # Display selected page content
        if page == "Data Overview":
            data_overview(df)
        elif page == "Column Analysis":
            column_filter(df)
        elif page == "Visualization":
            data_visualization(df)
        elif page == "Summary":
            data_summary(df)
        elif page == "Download":
            data_download(df)

# Entry point of the application
if __name__ == "__main__":
    main()